In [8]:
from .base import MagicStrings, load_ipython_extension, unload_ipython_extension

In [9]:
import base64, IPython

In [10]:
from collections import UserList

In [11]:
from vdom import *
from vdom.svg import *

In [12]:
element = div('%s', style={'flex': '1'})._repr_html_()
row = div('%s', style={'display': 'flex', 'flex-direction': 'row'})._repr_html_()
column = div('%s', style={'display': 'flex', 'flex-direction': 'column'})._repr_html_()

In [13]:
import mistune

In [22]:
from .utils import format_images

In [24]:
class Container(UserList):
    def _repr_html_(self, body=""):
        for object in self:
            output, metadata = IPython.get_ipython().display_formatter.format(object)
            for key in list(output):

                if key.startswith('image'):
                    encoded = output.pop(key)
                    if key.endswith('xml'):
                        output['text/html'] = encoded
                    else:
                        output.update(
                             IPython.get_ipython().display_formatter.format(
                                 format_images(key, output)
                            )[0])
            
            if 'text/markdown' in output:
                output['text/html'] = output.get('text/html', mistune.markdown(output.get(
                    'text/markdown'
                )))                                        
            body += element%output.get('text/html', output.get('text/plain', str(object)))                
        return {Row: row, Column: column}[type(self)]%body

    def __iter__(self):
        for object in super().__iter__(): yield ({Row, Column} - {type(self)}).pop()(object) if isinstance(object, list) else object

class Row(Container): "A flexbox row"
class Column(Container): "A flexbox column"

In [25]:
@MagicStrings.append('- *')
@MagicStrings.append('[[]*[]]')
def yaml(str):
    import yaml, io
    data = yaml.safe_load(io.StringIO(str))
    if str.startswith('[') and len(data) == 1:
        result = imports(str[1:-1])
        if result: return result
    return Row(data)

In [26]:
def imports(str):
    if len(str) > 2 and str[0] == str[1] == '`': return
    try:
        return __import__(str)
    except:...
        
    if hasattr(__import__('__main__'), str):
        return getattr(__import__('__main__'), str)
    
    if str in sys.modules:
        return sys.modules[str] 